# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 12:47PM,261421,10,HH-41410,Hush Hush,Released
1,Apr 19 2023 12:47PM,261421,10,HH-41411,Hush Hush,Released
2,Apr 19 2023 12:47PM,261421,10,HH-41413,Hush Hush,Released
3,Apr 19 2023 12:46PM,261422,10,EYE43047,Eye Pharma Inc,Released
4,Apr 19 2023 12:45PM,261420,10,9395428,Eye Pharma Inc,Released
5,Apr 19 2023 12:40PM,261418,10,Eme-112602,Emerginnova,Released
6,Apr 19 2023 12:38PM,261417,10,131136,"Virtue Rx, LLC",Released
7,Apr 19 2023 12:38PM,261417,10,9396013,"Virtue Rx, LLC",Released
8,Apr 19 2023 12:01PM,261415,19,ELI04202023,Eli Lilly and Company,Released
9,Apr 19 2023 11:58AM,261414,19,TR-RIH-04202023,"GCH Granules USA, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,261417,Released,2
26,261418,Released,1
27,261420,Released,1
28,261421,Released,3
29,261422,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261417,NaN,NaN,2.0
261418,NaN,NaN,1.0
261420,NaN,NaN,1.0
261421,NaN,NaN,3.0
261422,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260869,0.0,7.0,26.0
260910,0.0,9.0,4.0
261117,0.0,9.0,1.0
261321,0.0,13.0,12.0
261343,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260869,0,7,26
260910,0,9,4
261117,0,9,1
261321,0,13,12
261343,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260869,0,7,26
1,260910,0,9,4
2,261117,0,9,1
3,261321,0,13,12
4,261343,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260869,,7,26
1,260910,,9,4
2,261117,,9,1
3,261321,,13,12
4,261343,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 12:47PM,261421,10,Hush Hush
3,Apr 19 2023 12:46PM,261422,10,Eye Pharma Inc
4,Apr 19 2023 12:45PM,261420,10,Eye Pharma Inc
5,Apr 19 2023 12:40PM,261418,10,Emerginnova
6,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC"
8,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company
9,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc."
10,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health
11,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC
20,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 19 2023 12:47PM,261421,10,Hush Hush,,,3
1,Apr 19 2023 12:46PM,261422,10,Eye Pharma Inc,,,1
2,Apr 19 2023 12:45PM,261420,10,Eye Pharma Inc,,,1
3,Apr 19 2023 12:40PM,261418,10,Emerginnova,,,1
4,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",,,2
5,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,,,1
6,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",,1,
7,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,,1,
8,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC,,4,5
9,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:47PM,261421,10,Hush Hush,3,,
1,Apr 19 2023 12:46PM,261422,10,Eye Pharma Inc,1,,
2,Apr 19 2023 12:45PM,261420,10,Eye Pharma Inc,1,,
3,Apr 19 2023 12:40PM,261418,10,Emerginnova,1,,
4,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2,,
5,Apr 19 2023 12:01PM,261415,19,Eli Lilly and Company,1,,
6,Apr 19 2023 11:58AM,261414,19,"GCH Granules USA, Inc.",,1,
7,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,,1,
8,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC,5,4,
9,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:47PM,261421,10,Hush Hush,3,,
1,Apr 19 2023 12:46PM,261422,10,Eye Pharma Inc,1,,
2,Apr 19 2023 12:45PM,261420,10,Eye Pharma Inc,1,,
3,Apr 19 2023 12:40PM,261418,10,Emerginnova,1,,
4,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:47PM,261421,10,Hush Hush,3.0,NaN,NaN
1,Apr 19 2023 12:46PM,261422,10,Eye Pharma Inc,1.0,NaN,NaN
2,Apr 19 2023 12:45PM,261420,10,Eye Pharma Inc,1.0,NaN,NaN
3,Apr 19 2023 12:40PM,261418,10,Emerginnova,1.0,NaN,NaN
4,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 12:47PM,261421,10,Hush Hush,3.0,0.0,0.0
1,Apr 19 2023 12:46PM,261422,10,Eye Pharma Inc,1.0,0.0,0.0
2,Apr 19 2023 12:45PM,261420,10,Eye Pharma Inc,1.0,0.0,0.0
3,Apr 19 2023 12:40PM,261418,10,Emerginnova,1.0,0.0,0.0
4,Apr 19 2023 12:38PM,261417,10,"Virtue Rx, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1829852,9.0,1.0,0.0
15,522687,26.0,20.0,7.0
19,1829752,22.0,26.0,1.0
20,1305712,36.0,30.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1829852,9.0,1.0,0.0
1,15,522687,26.0,20.0,7.0
2,19,1829752,22.0,26.0,1.0
3,20,1305712,36.0,30.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,9.0,1.0,0.0
1,15,26.0,20.0,7.0
2,19,22.0,26.0,1.0
3,20,36.0,30.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,15,Released,26.0
2,19,Released,22.0
3,20,Released,36.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,7.0,1.0,0.0
Executing,1.0,20.0,26.0,30.0
Released,9.0,26.0,22.0,36.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,7.0,1.0,0.0
1,Executing,1.0,20.0,26.0,30.0
2,Released,9.0,26.0,22.0,36.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,7.0,1.0,0.0
1,Executing,1.0,20.0,26.0,30.0
2,Released,9.0,26.0,22.0,36.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()